In [26]:
import time
from selenium import webdriver
import pandas as pd
import numpy as np 
import random
import re

In [27]:
data =  pd.read_csv('./markettrends.csv', dtype = {'state': 'str', 'sa3_name16': 'str','sa4_name16': 'str',
                                                   'postcode': 'str','state': 'str','property_type':'str'})
ndata = data.fillna({'Volume of new rental listings (1 month)': 0, 'Volume of sales (1 month)': 0})
ndata

,state,sa3_name16,sa4_name16,postcode,property_type,value_at_date,Volume of new rental listings (1 month),Hedonic Rental Yields,Hedonic Home Value Index,Volume of sales (1 month)
0,NT,Darwin City,Darwin,0800,Houses,1980-01-31,0.0,0.0000,154.6729,0.0
1,NT,Darwin City,Darwin,0800,Houses,1980-02-29,0.0,0.0000,154.6729,0.0
2,NT,Darwin City,Darwin,0800,Houses,1980-03-31,0.0,0.0000,154.6729,0.0
3,NT,Darwin City,Darwin,0800,Houses,1980-04-30,0.0,0.0000,154.6729,0.0
4,NT,Darwin City,Darwin,0800,Houses,1980-05-31,0.0,0.0000,154.6729,0.0
...,...,...,...,...,...,...,...,...,...,...
2848883,ACT,NaN,Australian Capital Territory,NaN,Units,2021-01-31,686.0,0.0556,104.3396,207.0
2848884,ACT,NaN,Australian Capital Territory,NaN,Units,2021-02-28,527.0,0.0552,105.0188,370.0
2848885,ACT,NaN,Australian Capital Territory,NaN,Units,2021-03-31,578.0,0.0552,105.7753,478.0
2848886,ACT,NaN,Australian Capital Territory,NaN,Units,2021-04-30,483.0,0.0546,107.0882,355.0


In [28]:
UniquePostcode = pd.DataFrame(ndata.postcode.unique())

In [29]:
Allpostcode= UniquePostcode.iloc[0:2633,0]


In [30]:
Allpostcode

0       0800
1       0810
2       0812
3       0815
4       0820
        ... 
2628    7466
2629    7467
2630    7468
2631    7469
2632    7470
Name: 0, Length: 2633, dtype: object

In [40]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/leiflyu/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60/chromedriver] found in cache
/var/folders/s1/vpd2prqd5fsc16qqsymrc9b00000gn/T/ipykernel_4189/795523883.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [41]:
df =pd.DataFrame(index = Allpostcode.index)

time0 = time.time()
for num,postcode in enumerate(Allpostcode):
    driver.get("https://www.google.com/webhp?hl=en&sa=X&ved=0ahUKEwjLiLeuxMjxAhWFILcAHXG1D28QPAgI")
    time.sleep(0.1*random.random())
    search_box = driver.find_element_by_name('q')
    search_box.send_keys('Australia Postcode %s Latitude and Longitude' %(postcode))
    search_box.submit()
    try:
        result = driver.find_element_by_xpath('//div[@class="bVj5Zb"]').text
    except:
        try:
            result = driver.find_element_by_xpath('//div[@class="Z0LcW"]').text
        except:
            print("No. %i webcrawler failed for region %s" % (num, postcode)) 
            continue
    resultsplit = result.split(" ")            
    df.loc[num,"Postcode"] = postcode
    df.loc[num,"Latitude"] = resultsplit[0]
    df.loc[num,"Longitutde"] = resultsplit[2]
    df.loc[num,"Latitudedir"] = resultsplit[1]
    df.loc[num,"Longtitude"] = resultsplit[3]
    print("No. %i webcrawler succeeded for region %s" % (num, postcode))    
time.sleep(1)
driver.quit()
print(time.time() - time0)
    


/var/folders/s1/vpd2prqd5fsc16qqsymrc9b00000gn/T/ipykernel_4189/835077137.py:7: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  search_box = driver.find_element_by_name('q')
/var/folders/s1/vpd2prqd5fsc16qqsymrc9b00000gn/T/ipykernel_4189/835077137.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  result = driver.find_element_by_xpath('//div[@class="bVj5Zb"]').text
/var/folders/s1/vpd2prqd5fsc16qqsymrc9b00000gn/T/ipykernel_4189/835077137.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  result = driver.find_element_by_xpath('//div[@class="Z0LcW"]').text


No. 0 webcrawler succeeded for region 0800
No. 1 webcrawler succeeded for region 0810
No. 2 webcrawler succeeded for region 0812
No. 3 webcrawler failed for region 0815
No. 4 webcrawler succeeded for region 0820
No. 5 webcrawler succeeded for region 0822
No. 6 webcrawler succeeded for region 0828
No. 7 webcrawler succeeded for region 0829
No. 8 webcrawler succeeded for region 0830
No. 9 webcrawler succeeded for region 0832
No. 10 webcrawler succeeded for region 0834
No. 11 webcrawler succeeded for region 0835
No. 12 webcrawler succeeded for region 0836
No. 13 webcrawler succeeded for region 0837
No. 14 webcrawler succeeded for region 0838
No. 15 webcrawler succeeded for region 0839
No. 16 webcrawler succeeded for region 0840
No. 17 webcrawler succeeded for region 0841
No. 18 webcrawler succeeded for region 0845
No. 19 webcrawler succeeded for region 0846
No. 20 webcrawler succeeded for region 0847
No. 21 webcrawler succeeded for region 0850
No. 22 webcrawler succeeded for region 0852
N

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=100.0.4896.75)
Stacktrace:
0   chromedriver                        0x0000000102653c34 chromedriver + 4406324
1   chromedriver                        0x00000001025ed290 chromedriver + 3986064
2   chromedriver                        0x000000010224e84c chromedriver + 190540
3   chromedriver                        0x0000000102231f74 chromedriver + 73588
4   chromedriver                        0x0000000102298ae4 chromedriver + 494308
5   chromedriver                        0x00000001022a5eb4 chromedriver + 548532
6   chromedriver                        0x0000000102272864 chromedriver + 338020
7   chromedriver                        0x0000000102619320 chromedriver + 4166432
8   chromedriver                        0x000000010262e748 chromedriver + 4253512
9   chromedriver                        0x0000000102632cbc chromedriver + 4271292
10  chromedriver   